In [ ]:
import os
# DECLARE HOW MANY GPUS YOU WISH TO USE. 
# KAGGLE ONLY HAS 1, BUT OFFLINE, YOU CAN USE MORE
#os.environ["CUDA_VISIBLE_DEVICES"]="0" #0,1,2,3 for four gpu

# VERSION FOR SAVING MODEL WEIGHTS
VER='0.1.0'

# IF VARIABLE IS NONE, THEN NOTEBOOK COMPUTES TOKENS
# OTHERWISE NOTEBOOK LOADS TOKENS FROM PATH
#LOAD_TOKENS_FROM = '../input/py-bigbird-v26'
LOAD_TOKENS_FROM = '../input/feedbackprize2021/';


# IF VARIABLE IS NONE, THEN NOTEBOOK TRAINS A NEW MODEL
# OTHERWISE IT LOADS YOUR PREVIOUSLY TRAINED MODEL
#LOAD_MODEL_FROM = '../input/py-bigbird-v26'
#TRAINED_MODEL_DIR = '/home/peng_sun2/s3result/feedback-prize-2021/model/baseline1_ner_roberta'
TRAINED_MODEL_DIR = '../input/feedbackprize2021/'

do_train=  True;
#LOAD_MODEL_FROM = None;

# IF FOLLOWING IS NONE, THEN NOTEBOOK 
# USES INTERNET AND DOWNLOADS HUGGINGFACE 
# CONFIG, TOKENIZER, AND MODEL
#DOWNLOADED_MODEL_PATH = '../input/py-bigbird-v26' 
#DOWNLOADED_MODEL_PATH = '/home/peng_sun2/s3result/feedback-prize-2021/pretrained_model/baseline1_ner'

token_path_to_save = '../input/feedbackprize2021/pretrained_roberta_token'
pretrained_model_path = '../input/feedbackprize2021/pretrained_roberta'
#pretrained_model_path = '../input/feedbackprize2021/baseline1_ner_roberta_pretrained_cpu'
config_path_to_save = '../input/feedbackprize2021/pretrained_roberta_config'
#DOWNLOADED_MODEL_PATH = None;
  
MODEL_NAME = 'allenai/longformer-base-4096'
from torch import cuda
config = {
    'model_name': MODEL_NAME,   
    'max_length': 1024,
    'train_batch_size':4,
    'valid_batch_size':1,
    'epochs':5,
    'n_fold':5,
    'scheduler': 'CosineAnnealingLR',
    'learning_rate': 2.5e-5,
    'weight_decay': 1e-6,
    'learning_rates': [2.5e-5, 2.5e-5, 2.5e-6, 2.5e-6, 2.5e-7],
    'max_grad_norm':10,
    'device': 'cuda' if cuda.is_available() else 'cpu',
    'seed': 2022,
    'min_lr': 1e-6,
    'T_max': 500
    #'device': 'cpu'
}

# THIS WILL COMPUTE VAL SCORE DURING COMMIT BUT NOT DURING SUBMIT
COMPUTE_VAL_SCORE = True
if len( os.listdir('../input/feedback-prize-2021/test') )>5:
      COMPUTE_VAL_SCORE = False

In [ ]:
from transformers import *
model_to_download = True;
if model_to_download:
    #os.mkdir('model')
    tokenizer = AutoTokenizer.from_pretrained(token_path_to_save)
    #tokenizer.save_pretrained(token_path_to_save)

    config_model = AutoConfig.from_pretrained(config_path_to_save) 
    config_model.num_labels = 15
    #config_model.save_pretrained(config_path_to_save)

    backbone = AutoModelForTokenClassification.from_pretrained(pretrained_model_path, 
                                                               config=config_model)
    #backbone.save_pretrained(pretrained_model_path)

In [ ]:
import torch
torch.cuda.empty_cache()
backbone.to(config['device'])

